In [ ]:
import os, sys
import numpy as np
import MDAnalysis as mda
from MDAnalysis.analysis.base import AnalysisFromFunction
from MDAnalysis.coordinates.memory import MemoryReader
from MDAnalysis.analysis.distances import distance_array
from collections import Counter

In [ ]:
bPythonScriptExport=False

In [ ]:
if bPythonScriptExport:
    import argparse
    parser = argparse.ArgumentParser(description='Analyse a system trajectory and match solvent molecules to cluster sites, '
                                                 'using sets of residues within X Angs as the site definitons, and '
                                                 'occupancy rates to rank water additions.',
                                                 formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('--top', type=str, dest='topologyFile', default='seg.psf',
                        help='The name of the protein structure file.')
    parser.add_argument('--trj', type=str, dest='trajectoryFile', default='sum.xtc',
                        help='The name of the trajectory file.')
    parser.add_argument('--clust', type=str, dest='clustDefFile', default='cluster_definitions.txt',
                        help='The name of the cluster definitions file. Can be fed from calculate_contact_persistence.py. '
                             'Although only resids are currently being read in, the expected file format is:\n'
                             '0 Intersection: <segid:resid1> <segid:resid2> ...\n'
                             '0 Union: <segid:resid1> <segid:resid2> ...\n'                       
                             '1 Intersection: <segid:resid1> <segid:resid2> ...\n'
                             '1 Union: <segid:resid1> <segid:resid2> ...\n'
                             '...\n')
    parser.add_argument('-n', '--n_clust', type=int, dest='numCluster', default=None,
                        help='Consider only the top N clusters read in the file. If not given, compute for all clusters read.')    
    parser.add_argument('-d', type=float, dest='distCutoff', default=3.5,
                        help='Maximum separation for pairs to be considered in contact. Needs to match the value defined for clustering.')
    parser.add_argument('--out', type=str, dest='outputPrefix', default='clustered',
                        help='A prefix for all output files generated.')
    parser.add_argument('--sel_site', type=str, dest='selectionTextSite', default='name OW and resname SOL',
                        help='An MDAnalysis selection text that refers to the atoms by which clustering was conducted. '
                       'Examples: "name CL" and "name OW and resname SOL".')    
    parser.add_argument('--sel_host', type=str, dest='selectionTextHost', default='protein',
                        help='An MDAnalysis selection text that chooses the host solute by which clustering was conducted.')    
    parser.add_argument('--sel_solvent', type=str, dest='selectionTextSolvent', default='resname SOL',
                        help='(Output) An MDAnalysis selection text that chooses the solvents to create a universe for.')
    parser.add_argument('--sel_system', type=str, dest='selectionTextSystem', default=None,
                        help='(Output) An MDAnalysis selection text that chooses atoms from the host system to include iin the output.'
                        'If not given, then only the cluster solvent atoms are written.')
    parser.add_argument('--in_mem', action='store_true', dest='bInMemory',
                        help='Asks MDAnlaysis to load the entire trajectory into memory.')
    args = parser.parse_args()
    
    #ratioCutoff  = args.ratioCutoff
    topFile      = args.topologyFile
    trjFile      = args.trajectoryFile
    clustDefFile = args.clustDefFile
    nClusts      = args.numCluster
    distCutoff   = args.distCutoff
    
    outputPrefix = args.outputPrefix

    seltxtSite   = args.selectionTextSite
    seltxtHost   = args.selectionTextHost
    seltxtSolv   = args.selectionTextSolvent
    seltxtSyst   = args.selectionTextSystem
    bInMemory    = args.bInMemory    
    MDAbackend   = 'serial'

In [ ]:
if not bPythonScriptExport:
    %cd ..
    allele='N1303K' ; temperature='310K' ; repl=3
    workDir = './trajectories/%s/%s/%i' % (allele, temperature, repl)
    topFile      = os.path.join(workDir, 'seg.psf')
    trjFile      = os.path.join(workDir, 'sum.xtc')
    clustDefFile = 'Stable_Solvent_Clustering.cluster_definitions_d3.5_r0.50.txt'
    distCutoff   = 3.5
    
    #outputPrefix = os.path.join(workDir, 'clustered')
    outputPrefix = os.path.join(workDir, 'clustered_d3.5_r0.50')
    
    seltxtSite   = "name OW"
    seltxtHost   = "protein"
    seltxtSolv   = "resname SOL"
    seltxtSyst   = "(protein or resname ATP MG) and not name MN? 1MN? 2MN? 1MC? 2MC?"
    bInMemory    = True
    nClusts      = 33
    MDAbackend   = 'openMP'

In [ ]:
# = = = Optional modules for profiling. Move to after argparse to prevent it from reporting when -h is invoked.
try:
    import psutil
    print( "= = psutil module installed. Will report memory use.")
    def get_memory_use():
        proc = psutil.Process(os.getpid())
        mem = proc.memory_info().rss/2**20
        print( "= = Current memory footprint @ %s : %.3f MiB" % (time.ctime(), mem) )
except ModuleNotFoundError:
    print( "= = psutil module not installed. Will not report memory use.")
    def get_memory_use():
        return
try:
    from tqdm.auto import tqdm
    print( "= = tdqm module installed. Will report loop progress.")
except ModuleNotFoundError:
    print( "= = tdqm module not installed. Will not report loop progress.")    
    def tdqm(f):
        return f

import time

In [ ]:
def process_list_items( ll ):
    for l in ll:
        l[0].sort()
        for item in l[0]:
            if item in l[1]:
                l[1].remove(item)
        l[1].sort()
    
#0 Intersection: ND1:573 ND1:603 ND1:464 ND1:465 ND1:572
#0 Union: ND1:573 ND1:603 ND1:464 ND1:465 ND1:572 ND2:1348 ND1:493
#1 Intersection: ND1:549 ND2:1251 ND1:550
#1 Union: ND2:1370 ND1:549 ND1:551 ND2:1371 ND1:553 ND1:548 ND2:1251 ND1:550 ND2:1291
#2 Intersection: LAS:28 TD2:1032 TD2:1036 TD2:1033 LAS:24
#2 Union: TD2:1037 TD2:1032 TD2:1036 LAS:27 LAS:24 LAS:28 TD2:1034 TD2:1033 TD2:1035 LAS:25                                                                    3
#...
def read_cluster_definitions(fileName):
    out=[]
    items=[]    
    with open(fileName,'r') as fp:
        for line in fp:
            l=line.split()
            if len(l)<2:
                continue
            elif len(l)==2:
                items.append([])
            else:
                temp=[]
                for i in range(2,len(l)):
                    temp.append( int(l[i].split(':')[-1]) )
                items.append(temp)
            if len(items)==2:
                out.append(items)
                items=[]
    return out

In [ ]:
def parse_list(l, pref="", suff="", sep=" "):
    out=""
    for item in l:
        out+="%s%s%s" % (pref, item, suff)
        if item != l[-1]:
            out+=sep
    return out

def write_cluster_selection_text(l, selSite="name OW", selSolute="protein", d=5.0):
    out=[]
    for i in l:
        s1=parse_list(i[0], pref="(around %g (resid " % d, suff=" and %s))" % selSolute, sep=" and ")
        if s1 !="":
            s1+=" and "
        s2 = parse_list(i[1], pref="", sep=" ")
        s2b = "(around %g (resid %s and protein))" % (d, s2)
        out.append("%s and %s %s" % (selSite, s1, s2b))
    return out

In [ ]:
def load_subuniverse_inmem(u, selectionText):
    a = u.select_atoms(selectionText)
    return mda.Merge(a).load_new(AnalysisFromFunction(lambda ag: ag.positions.copy(), a).run().results, format=MemoryReader)

In [ ]:
selectionResids = read_cluster_definitions(clustDefFile)
process_list_items(selectionResids) 
stringSelectionResids = [ [str(s[0]).strip('[]'),str(s[1]).strip('[]')] for s in selectionResids ]
x = write_cluster_selection_text(selectionResids, selSite=seltxtSite, selSolute=seltxtHost, d=distCutoff)
print("= = cluster definition file processed with distance cutoff %g Angs." % distCutoff)
print("   ...selection text for cluster 0:", x[0])

if nClusts is None:
    nClusts = len(selectionResids)

In [ ]:
bWriteOnlyWaters=True
if seltxtSyst is not None:
    bWriteOnlyWaters=False

In [ ]:
bInMemory=True

In [ ]:
u = mda.Universe(topFile,trjFile, in_memory=False)
nFrames=u.trajectory.n_frames
if bInMemory:
    print("= = Loading the trajectories into memory for subselections %s and %s ..." % (seltxtHost, seltxtSite) )
    # The in_memory version loads just the protein and solvent atoms for computation of distance matrices
    uHost  = load_subuniverse_inmem(u, seltxtHost)
    print("= = Loaded host sub-universe with %i atoms. Memory of loaded trajectory: %i MiB" % (uHost.trajectory.n_atoms, sys.getsizeof(uHost.trajectory.coordinate_array)/2**20 ) )
    uSite  = load_subuniverse_inmem(u, seltxtSite)
    print("= = Loaded site sub-universe with %i atoms. Memory of loaded trajectory: %i MiB" % (uSite.trajectory.n_atoms, sys.getsizeof(uSite.trajectory.coordinate_array)/2**20 ) )
get_memory_use()

In [ ]:
if bInMemory:
    indexSelectionResids = []
    for selClust in selectionResids:
        # Each residue in the intersect selection needs its own list of indices for slicing.
        # The whole residue in the union selection is added in a single set of incides for slicing.
        temp = [ uHost.select_atoms("resid %i" % r).indices for r in selClust[0] ]
        if len(selClust[1])>0:
            temp.append( uHost.select_atoms("resid %s" % ' '.join(map(str,selClust[1])) ).indices )
        indexSelectionResids.append( temp )
else:
    listAtomSels = []
    for i in range(nClusts):
        listAtomSels.append( u.select_atoms(x[i], updating=True, periodic=False) )

In [ ]:
atomSelSite=u.select_atoms(seltxtSite)
nSites = atomSelSite.n_atoms
dictGlobalIndex = { a:b for a,b in enumerate(atomSelSite.indices) }
drevGlobalIndex = { b:a for a,b in dictGlobalIndex.items()}    

In [ ]:
"""
The matBoolIndices holds the information about which water ID is within the cutoffdistance from the protein residue atoms.
One can access the fram and index information via np.where( matBoolIndices[frame,cluster,waterID] )
"""
print("= = Allocating large boolean matrix for fast computation.")
matBoolIndices=np.full( (nFrames, nClusts, nSites), False)
matTemps=[]
for i in range(nClusts):
    nSel = len(indexSelectionResids[i])
    matTemps.append( np.full( (nSel, nSites), False) )

# = = = Profile memory use now.
get_memory_use()
    
if bInMemory:
    print("= = Entering lengthy distance matrix computation to determine water site occupancies...")
    print("    ...Starting at: %s" % time.ctime() )        
    for f in tqdm(range(nFrames)):
        for c in range(nClusts):
            nSel = len(indexSelectionResids[c])
            #matTemp = np.full( (nSel, uSite.n_atoms), False)
            matTemps[c][:] = False
            # = = Get site atoms within X of any host atoms in sub-selection, e.g. any atom of protein resid X
            for j in range(nSel):
                posHost=uHost.trajectory.coordinate_array[f,indexSelectionResids[c][j]]
                distMat = mda.analysis.distances.distance_array(posHost,
                                                                uSite.trajectory.coordinate_array[f],
                                                                backend=MDAbackend)
                matTemps[c][j] = np.any( distMat<distCutoff, axis=0 )
            # = = Filter only site atoms that are within the cutoff or all selections.
            matBoolIndices[f,c] = np.all( matTemps[c], axis=0 )
else:
    print("= = Entering lengthy distance matrix computation to determine water site occupancies...")
    print("    ...Starting at: %s" % time.ctime() )
    for f in tqdm(range(nFrames)):
        u.trajectory[f]
        for c in range(nClusts):
            for i in listAtomSels[c].indices:
                matBoolIndices[f,c,drevGlobalIndex[i]]=True
    print("    ...Ending at: %s" % time.ctime() )    

In [ ]:
print( "= = Boolean matrix of site occupancies complete. Cleaning up sub-universes (if applicable).")
if bInMemory:
    # = = Deallocation of the three largest matrices to save memory
    matTemps = None 
    uHost.trajectory = None ; uSite.trajectory = None
get_memory_use()

In [ ]:
from bokeh.io import output_notebook, output_file, show
from bokeh.plotting import figure
from bokeh.layouts import row, column
import bokeh.models as bokehModels
import bokeh.palettes as bokehPalettes

def map_palette(p, v, vMin, vMax):
    x=int((v-vMin)/(vMax-vMin)*(len(p)-1))
    return p[x]

def create_JS_update_visibility(listGlyphs, div="", divText=None):
    return bokehModels.CustomJS(args=dict(gs=listGlyphs, d = div, dt=divText),
             code="""
             //var s = this.item;
             var s = this.value_throttled;
             for (var i = 0; i < gs.length; i++) {
                 if ( i == s ) {
                     gs[i].visible = true
                 } else if ( gs[i].visible ) {
                     gs[i].visible = false
                 }
             }
             if ( d != "" ) {
                 d.text = '<strong>Core resids:</strong> ' + dt[s][0] + '<br><strong>Peripheral resids:</strong> ' + dt[s][1]
             }
             """
            )

def export_site_occupancy_as_bokehHTML(outputFile,
                                       listXs, listYs, listColours,
                                       listClusterDefs=None,
                                       titleText='Water occupancy',
                                       bNoteBook=False,
                                       palette=bokehPalettes.Viridis256
                                      ):
    nPlots = len(listXs)
            
    fig = figure(title=titleText, plot_width=800, plot_height=400,
                 x_axis_label='Trajectory frame', y_axis_label='solvent index')
    listSources = []
    listGlyphs   = []
    for i in range(nPlots):       
        source = bokehModels.ColumnDataSource({'x': listXs[i], 'y' : listYs[i],
                                               'color': listColours[i],
                                              })
        glyph = fig.scatter(x = 'x', y = 'y', size=2, color='color', source = source)          
        listSources.append(source)
        glyph.visible=(i==0)
        listGlyphs.append(glyph)

    colourMapper = bokehModels.mappers.LinearColorMapper(palette=palette, low=0.0, high=1.0)
    colourBar = bokehModels.ColorBar(title='Occupancy rate',
                                     color_mapper=colourMapper, label_standoff=12)
    fig.add_layout(colourBar, 'above')

    if listClusterDefs is None:
        div = bokehModels.Div(width=600, text=trjFile)
    else:
        div = bokehModels.Div(width=600, text='<strong>Core resids:</strong> %s<br><strong>Peripheral resids:</strong> %s' % \
                              (listClusterDefs[0][0], listClusterDefs[0][1]))
        
    sliderWidget = bokehModels.Slider(width=200,title='Show cluster...', start=0, end=nClusts-1, step=1, value=0)
    sliderWidget.js_on_change('value_throttled',
                              create_JS_update_visibility(listGlyphs, div, listClusterDefs))

    output_file(outputFile)
    if not bPythonScriptExport:
        output_notebook()
    show(column(row(sliderWidget,div),fig))            

In [ ]:
listCounters=[]
for i in range(nClusts):
    c =  Counter( np.where(matBoolIndices[:,i])[1] )
    print( "Cluster %i, 5 most common waters:" % i, c.most_common(5) )
    listCounters.append( c )

In [ ]:
listAtomColours=[]
for i in range(nClusts):
    #c=listDictAtomData[i]['counter']
    c=listCounters[i]
    l=[]
    for index in np.where(matBoolIndices[:,i])[1]:
         l.append( map_palette( bokehPalettes.Viridis256, c[index], 0, nFrames )  )
    listAtomColours.append( l )

export_site_occupancy_as_bokehHTML( outputPrefix+'_site_occupancy_all.html',
                                   [ np.where(matBoolIndices[:,i])[0] for i in range(nClusts) ],
                                   [ np.where(matBoolIndices[:,i])[1] for i in range(nClusts) ],
                                   [ listAtomColours[i] for i in range(nClusts) ],
                                   listClusterDefs=stringSelectionResids,
                                   titleText='Waters satisfying consensus cluster residues sets, using distance cutoff %g Angs.' % distCutoff,
                                   bNoteBook=not bPythonScriptExport,
                                   palette=bokehPalettes.Viridis256
                                   )

## Populate the assignment of a water molecule to every site

In [ ]:
def count_duplicate_assignments(arr, emptyID=0):
    d=[]
    for j in range(arr.shape[1]):
        check = arr[np.nonzero(arr[...,j]-emptyID)[0],j]
        if len(np.unique(check))<len(check):
            d.append(j)    
    print( "...Debug: frames with duplicate assignments:", len(d) )

In [ ]:
def report_current_cluster_allocations(clusterAssignments, emptyID=-1):
    sizeTotal = clusterAssignments.shape[0]*clusterAssignments.shape[1]
    sizeAllocated = np.count_nonzero(clusterAssignments-emptyID)
    print ( "Assigned : missing : Total" )
    print ( sizeAllocated, sizeTotal - sizeAllocated, sizeTotal )

In [ ]:
# The complete non-redundant assignment of water molecules to each cluster for each frame.
emptyID=-1
clusterAssignments=np.zeros((nClusts,nFrames), dtype=int)
if emptyID != 0:
    clusterAssignments[:]=emptyID

# List of waters to be ignored as they are adjacent clusters.
listAssigned=[]

# Copy of counter lists to pop and modify as necessary.
cTest=[]
for i in range(nClusts):
    cTest.append( [ (k,v) for k,v in listCounters[i].most_common() ] )

### Step 1.
Assign waters with maximium occupancy in each cluster. Order by clusters with higher prevalence among simulations (lowID to highID).

In [ ]:
for i in range(nClusts):
    for k, v in cTest[i]:
        cTest[i].pop(0)
        if k in listAssigned:
            print("Water %i repeated in cluster %i" % (k,i))
            continue
        listAssigned.append(k)
        # = = = Assign the frames in where water ID k is found in cluster i
        clusterAssignments[ i, np.where( matBoolIndices[:,i,k] )[0] ] = k
        #for j in range(nFrames):
        #    if k in listDictAtomData[i]['indices'][j]:
        #        clusterAssignments[i,j] = k
        print("Water %i (occ. %i) assigned to cluster %i by maximum occupancy" % (k,v,i))
        break

rAssigned=100*np.count_nonzero(clusterAssignments-emptyID)/(nClusts*nFrames)
print("   ...%.1f%% assigned so far." % rAssigned)

In [ ]:
count_duplicate_assignments(clusterAssignments, emptyID=-1)
report_current_cluster_allocations(clusterAssignments, emptyID=-1)

### Step 2.
Starting populating the assignment matrix by working through the cluster-specific counters, ordered by highest occupancy.

In [ ]:
cTest2 = np.array([ [i,k,v] for i,l in enumerate(cTest) for k,v in l ])
cTest2 = cTest2[ np.flip(np.argsort(cTest2[...,2])) ]

In [ ]:
minFrameBreak=0.01*nFrames
#minFrameBreak=1
for cID, k, v in cTest2:
    if v<minFrameBreak:
        break
    if k in listAssigned:
        #print("Water %i repeated in cluster %i" % (k,cID))
        continue
        
    # = = = Assign the frames in where water ID k is found in cluster i
    # clusterAssignments[ cID, np.where( matBoolIndices[:,cID,k] )[0] ] = k 
    # Two checks are performed.
    # - The ==emptyID checks that cluster assignments is currenty empty.
    # - The np.any() checks that water ID k hasn't been assigned to another cluster yet in that same frame.
    indices = np.where(np.logical_and(np.logical_and(matBoolIndices[:,cID,k],
                                                     clusterAssignments[cID] == emptyID),
                                                     np.all(clusterAssignments!=k,axis=0)
                      ))[0]
    clusterAssignments[cID, indices ] = k
    #countOverlap=0    
    #for j in range(nFrames):
    #    if k in clusterAssignments[...,j]:
    #        # = = = Preassigned to another cluster
    #        continue
    #    if k in listDictAtomData[cID]['indices'][j]:
    #        if clusterAssignments[cID,j]==emptyID:
    #            clusterAssignments[cID,j] = k
    #        else:
    #            countOverlap+=1
    #if countOverlap<v:
    #    print("Water %i (n: %i of %i) assigned to cluster %i by sequential fill. " % (k,v-countOverlap,v,cID))
    
rAssigned=100*np.count_nonzero(clusterAssignments-emptyID)/(nClusts*nFrames)
print("   ...%.1f%% assigned so far." % rAssigned)

In [ ]:
count_duplicate_assignments(clusterAssignments, emptyID=-1)
report_current_cluster_allocations(clusterAssignments, emptyID=-1)

### Step 3.
After this, extend empty assignments by looking at existing populated entries on either side. Now ignoring whether the assignment is already taken.

In [ ]:
def return_discrete_ranges(arr):
    if len(arr)==0:
        return [],[]
    left=[] ; right=[]
    l=arr[0] ; r=arr[0]
    for v in arr[1:]:
        if v==r+1:
            r+=1
            continue
        left.append(l) ; right.append(r)
        l=v ; r=v
    left.append(l) ; right.append(r)
    return left, right

# = = = Fill in voids from existing water IDs, based on adjacency.
for i in range(nClusts):
    l,r = return_discrete_ranges( np.where(clusterAssignments[i]==emptyID)[0] )
    for a,b in zip(l,r):        
        if a==0:
            if b==nFrames-1:
                # Completely empty
                continue
            else:
                clusterAssignments[i,a:b+1]=clusterAssignments[i,b+1]
        elif b==nFrames-1:
            clusterAssignments[i,a:b+1]=clusterAssignments[i,a-1]
        else:
            m=int(0.5*(a+b))
            clusterAssignments[i,m:b+1]=clusterAssignments[i,b+1]            
            clusterAssignments[i,a:m]=clusterAssignments[i,a-1]   

In [ ]:
count_duplicate_assignments(clusterAssignments, emptyID=-1)
report_current_cluster_allocations(clusterAssignments, emptyID=-1)

In [ ]:
# = = = Cleanup.
matBoolIndices = None

In [ ]:
print( "= = Cluster assignments complete. Cleaning up boolean matrix.")
get_memory_use()

### Plot the final assignment, taking note of duplicate assignemnts due to the final step.

In [ ]:
colours = np.chararray(clusterAssignments.shape, itemsize=7)
for j in range(clusterAssignments.shape[1]):
    for i in range(clusterAssignments.shape[0]):
        if clusterAssignments[i,j]==emptyID:
            colours[i,j]='#4422FF'
            continue
        if np.count_nonzero(clusterAssignments[:,j]==clusterAssignments[i,j])>1:
            colours[i,j]='#FF4422'
        else:
            colours[i,j]='#000000'
colours = colours.astype(str)

In [ ]:
export_site_occupancy_as_bokehHTML(outputPrefix+'_site_occupancy_filled.html',
                                   np.tile( np.arange(nFrames), (nClusts,1) ),
                                   clusterAssignments,
                                   colours,
                                   listClusterDefs=stringSelectionResids,
                                   titleText='Final Waters assigned to cluster, using distance cutoff %g Angs.' % distCutoff,
                                   bNoteBook=not bPythonScriptExport,
                                   palette=bokehPalettes.Viridis256
                                   )

## Writing Output system
Note: Completely empty assignments will remain empty from this stage onwards.

### Convert the solvent indices back to global indices.

In [ ]:
# = = = Load the positions of all waters at once to save on File-IO
# = = = This one is too expensive in terms of getting *every water*
if False:
    uAllWatersIn = load_subuniverse_inmem(u, seltxtSolv)
    atomSelSite  = uAllWatersIn.select_atoms(seltxtSite)
    dictGlobalIndex = { a:b for a,b in enumerate(atomSelSite.indices) }
    print("= = Created subuniverse for all waters.")
    get_memory_use()

k = np.array(list(dictGlobalIndex.keys()))
v = np.array(list(dictGlobalIndex.values()))
mapping_ar = np.zeros(k.max()+1,dtype=v.dtype) #k,v from approach #1
mapping_ar[k] = v
    
clusterAssignmentsOut = mapping_ar[clusterAssignments]

In [ ]:
def create_water_universe(nSol, listNames=['O', 'H', 'H'], resName='SOL', res0=1, segID='SOL', bTraj=False):
    nAtoms = nSol*3
    resIndices = np.repeat(range(nSol), 3)
    segIndices = [0]*nSol
    sol = mda.Universe.empty(nAtoms, n_residues=nSol, atom_resindex=resIndices, residue_segindex=segIndices, trajectory=bTraj)
    sol.add_TopologyAttr('name', listNames*nSol)
    sol.add_TopologyAttr('type', ['O', 'H', 'H']*nSol)
    sol.add_TopologyAttr('resname', [resName]*nSol)
    sol.add_TopologyAttr('resid', list(range(res0,nSol+res0)))
    sol.add_TopologyAttr('segid', [segID])
    
    bonds = []
    for i in range(0, nAtoms, 3):
        bonds.extend([(i, i+1), (i, i+2)])
    sol.add_TopologyAttr('bonds', bonds)
    return sol

def create_universe_from_selection(u, atomSel):
    """
    Need an In-memory version in order to write-out the trajectory
    """
    from MDAnalysis.analysis.base import AnalysisFromFunction
    xyz = AnalysisFromFunction(lambda ag: ag.positions.copy(),
                                   atomSel).run().results
    u2 = mda.Merge(atomSel)
    u2.load_new(xyz, format=MemoryReader)

def get_water_site_positions(mdaU, oxyIDs, emptyID=0):
    nSol, nFrames = oxyIDs.shape
    posOut=np.zeros( (nFrames,nSol*3,3), dtype=float)
    for j in range(nFrames):
        mdaU.trajectory[j]
        for i in range(nSol):
            oxyID = oxyIDs[i,j]
            if oxyID > emptyID:
                posOut[j,i*3:(i+1)*3,:]=mdaU.select_atoms("index %i %i %i" % (oxyID,oxyID+1,oxyID+2)).positions
    return posOut


In [ ]:
# = = = Create water universe from positions. This one is too expensive in terms of getting *every water*
print( "= = Creating a pure solvent universe for writing.")
if False:
    memReader = MemoryReader(get_water_site_positions(uAllWatersIn, clusterAssignmentsOut),
                         dt=u.trajectory.dt)
    uWat = create_water_universe(nClusts, listNames=['OW', 'HW1', 'HW2'], res0=0, segID='CRY', bTraj=True)
    uWat.trajectory = memReader
    uAllWatersIn.trajectory = None
else:
    memReader = MemoryReader(get_water_site_positions(u, clusterAssignmentsOut),
                         dt=u.trajectory.dt)
    uWat = create_water_universe(nClusts, listNames=['OW', 'HW1', 'HW2'], res0=0, segID='CRY', bTraj=True)
    uWat.trajectory = memReader        

print( "= = Pure solvent universe created.")
get_memory_use()
# Use mda.Merge() as needed to write

In [ ]:
print( "= = Writing the output universe to file...")
if bWriteOnlyWaters:
    selWatersOut = uWat.select_atoms("all")
    selWatersOut.universe.trajectory[0]
    selWatersOut.write(outputPrefix+'_solvent.pdb')   
    selWatersOut.write(outputPrefix+'_solvent.xtc', frames='all')    
else:
    # Create a new universe from the combination of the two. 
    selSoluteIn  = u.select_atoms(seltxtSyst)
    selWatersIn  = uWat.select_atoms("all")
    uSystemOut   = mda.Merge(selSoluteIn,selWatersIn)
    nAtomsSolute = selSoluteIn.n_atoms
    nAtomsWater  = selWatersIn.n_atoms
    dimSystemOut = (nFrames, uSystemOut.trajectory.n_atoms, 3 )
    memReaderOutputSystem = MemoryReader(np.zeros( dimSystemOut, dtype=float ), dt=u.trajectory.dt)
    uSystemOut.trajectory = memReaderOutputSystem
    
    # = = = Transfer positions to the in-memory output system from the source system (which is probably from file)
    if False:
        uSoluteIn = load_subuniverse_inmem(u, seltxtSyst)
        uSystemOut.trajectory.coordinate_array[:,0:nAtomsSolute,:] = uSoluteIn.trajectory.coordinate_array
        uSystemOut.trajectory.coordinate_array[:,nAtomsSolute:nAtomsSolute+nAtomsWater,:] = uWat.trajectory.coordinate_array
    else:
        selSoluteOut = uSystemOut.select_atoms(seltxtSyst)
        selWatersOut = uSystemOut.select_atoms(seltxtSolv)
        for f in range(uSystemOut.trajectory.n_frames):
            u.trajectory[f] ; uWat.trajectory[f] ; uSystemOut.trajectory[f]
            selSoluteOut.positions = selSoluteIn.positions
            selWatersOut.positions = selWatersIn.positions

    selSystemOut = uSystemOut.select_atoms("all")
    selSystemOut.universe.trajectory[0]
    selSystemOut.write(outputPrefix+'.pdb')
    selSystemOut.write(outputPrefix+'.xtc', frames='all')

In [ ]:
print( "= = Trajectory writing complete. Finishing up.")
get_memory_use()